In [1]:
from datetime import date, datetime, time, timedelta
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta

## Table of Contents
1. `datetime`'s date, datetime, time and timedelta. Timezones with `pytz`
2. using `dateutil`'s `relativedelta` for shifting datetimes by weeks, months, quarters and years
3. dates and times in pandas
4. pandas `Timestamp` conversions of `timezone`s

### 1. `datetime`'s date, datetime, time and timedelta. Timezones with `pytz`

#### 1.1. basic instantiation

In [2]:
d = date(2023, 11, 4)
dt = datetime(2023, 12, 6, 10, 11, 23)
print(dt.tzinfo)

None


#### 1.2. listing all relevant `pytz` timezones

In [3]:
pytz.all_timezones[:5]

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara']

#### 1.3. instantiating `pytz` timezones: CONTINENT/CITY format

In [4]:
WAWTZ = pytz.timezone("Europe/Warsaw")
MOSTZ = pytz.timezone("Europe/Moscow")
BERTZ = pytz.timezone("Europe/Berlin")

#### 1.4. NAIVE and AWARE `datetime`s: basic operations

In [5]:
# set timezone on a datetime
print(dt)
dt = dt.astimezone(WAWTZ)
print(dt)

2023-12-06 10:11:23
2023-12-06 10:11:23+01:00


In [6]:
dt.astimezone(MOSTZ)

datetime.datetime(2023, 12, 6, 12, 11, 23, tzinfo=<DstTzInfo 'Europe/Moscow' MSK+3:00:00 STD>)

##### `datetime` without timezone information - naive `datetime`

In [7]:
naive_dt = datetime(2023, 11, 22, 16, 44, 12)
print(naive_dt)
# member field with tz information: .tzinfo
print(naive_dt.tzinfo)

2023-11-22 16:44:12
None


##### making a `datetime` timezone-aware without re-casting it in different timezone 

In [8]:
aware_dt_warsaw = naive_dt.replace(tzinfo=WAWTZ)
print(aware_dt_warsaw)

2023-11-22 16:44:12+01:24


##### it is 16:44 in Warsaw. Сколько сейчас времени в Москве?

In [9]:
aware_dt_moscow = aware_dt_warsaw.astimezone(MOSTZ)
print(aware_dt_moscow)

2023-11-22 18:20:12+03:00


In [10]:
print(f"It is {aware_dt_warsaw} in Warsaw and {aware_dt_moscow} in Moscow. ")

It is 2023-11-22 16:44:12+01:24 in Warsaw and 2023-11-22 18:20:12+03:00 in Moscow. 


##### last but not least, how to get rid of timezone information from aware datetime?

In [11]:
aware_dt_warsaw = aware_dt_warsaw.replace(tzinfo=None)
print(aware_dt_warsaw)

2023-11-22 16:44:12


### 2. using `dateutil`'s `relativedelta` for shifting datetimes by weeks, months, quarters and years

Important: distinct between singular and plural arguments (source - docs: https://dateutil.readthedocs.io/en/stable/relativedelta.html):

* year, month, day, hour, minute, second, microsecond:
    Absolute information (argument is singular); adding or subtracting a
    relativedelta with absolute information does not perform an arithmetic
    operation, but rather REPLACES the corresponding value in the
    original datetime with the value(s) in relativedelta.
* years, months, weeks, days, hours, minutes, seconds, microseconds:
    Relative information, may be negative (argument is plural); adding
    or subtracting a relativedelta with relative information performs
    the corresponding arithmetic operation on the original datetime value
    with the information in the relativedelta.

#### 2.1. shifting date by one week

In [12]:
d = date(2024, 1, 29)
d_special = date(2024, 1, 31)

In [13]:
dpW1 = d + relativedelta(weeks=1)
print(dpW1)

2024-02-05


#### 2.2. shifting date by months

In [14]:
dpM1 = d + relativedelta(months=1)
print(dpM1)
d_special_pM1 = d + relativedelta(months=1)
print(d_special_pM1)

2024-02-29
2024-02-29


#### 2.3. shifting date by quarters

#### 2.4. shifting date by years

In [15]:
dpY1 = d + relativedelta(year=1)
print(dpY1)  # replaces
dpYs1 = d + relativedelta(years=1)
print(dpYs1)  # adds to

0001-01-29
2025-01-29


## 3. dates and times in pandas

* `pd.to_datetime` - convert string, date, datetime columns to timestamp columns (`np.datetime64`)
* pandas `dt` accessor works on `Timestamp` (aka `np.datetime64`) columns only

#### 3.0. type of column 

In [51]:
pd.to_datetime(date(2023, 1, 1))

Timestamp('2023-01-01 00:00:00')

In [53]:
pd.to_datetime(datetime(2023, 1, 2, 13, 0))

Timestamp('2023-01-02 13:00:00')

In [78]:
data = pd.DataFrame(data={"date": [date(2024, 1, 4), date(2024, 1, 2)]})
data["date"] = pd.to_datetime(data["date"])

In [82]:
print(data["date"][1])
print(data["date"][1].value)

2024-01-02 00:00:00
1704153600000000000


#### 3.1. `date` to `np.datetime64` with `pd.to_datetime`

In [42]:
df = pd.DataFrame(data={"date": [date(2023, 1, 10), date(2023, 1, 12)]})
df

,date
0,2023-01-10
1,2023-01-12


In [43]:
df.dtypes

date    object
dtype: object

In [44]:
# note that date object is stored as object type in a data frame
df["date"][0]

datetime.date(2023, 1, 10)

In [54]:
temp = pd.to_datetime(df["date"])
temp

0   2023-01-10
1   2023-01-12
Name: date, dtype: datetime64[ns]

#### 3.2. `str` to `np.datetime64` with `pd.to_datetime`

In [32]:
df["date"] = df["date"].apply(lambda x: x.strftime("%Y-%m-%d"))

In [33]:
df["date"][0]

'2023-01-10'

In [47]:
pd.to_datetime(df["date"])

0   2023-01-10
1   2023-01-12
Name: date, dtype: datetime64[ns]

#### 3.3. `datetime` to `np.datetime64` with `pd.to_datetime`

In [49]:
df["date"][0]

datetime.date(2023, 1, 10)

## 4. pandas `Timestamp` conversions of `timezone`s

In [57]:
data = pd.DataFrame(data={"date": ["2023-01-02", "2023-01-03"]})

In [62]:
data["date"] = pd.to_datetime(data["date"])
data.dtypes

date    datetime64[ns]
dtype: object

In [63]:
data["date"][0]

Timestamp('2023-01-02 00:00:00')

#### 4.1. `tz_localize` method

In [67]:
data["date_UTC"] = data["date"].dt.tz_localize(pytz.UTC)

#### 4.2. `tz_convert` method

In [71]:
CET = pytz.timezone("CET")
data["date_CET"] = data["date_UTC"].dt.tz_convert(CET)
data

,date,date_UTC,date_CET
0,2023-01-02,2023-01-02 00:00:00+00:00,2023-01-02 01:00:00+01:00
1,2023-01-03,2023-01-03 00:00:00+00:00,2023-01-03 01:00:00+01:00
